In [1]:
# import os

# from google.colab import drive

# drive.mount('/content/gdrive', force_remount=True)

# %cd "/content/gdrive/My Drive/Colab Notebooks"

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


!pip install -r requirement.txt


In [1]:
import os
import torch
import argparse
import itertools
import matplotlib.pyplot as plt
from typing import Optional

from FinanceSigWGANBase.evaluate import evaluate_generator
from lib.networks import get_generator
from lib.utils import to_numpy, load_obj
from lib.augmentations import parse_augmentations
from lib.test_metrics import get_standard_test_metrics
from lib.trainers.sig_wgan import compute_expected_signature

from FinanceSigWGANBase.utils.trainer import SigWGANTrainer
from FinanceSigWGANBase.utils.plot import plot_signature, plot_test_metrics
from FinanceSigWGANBase.utils.datasets import get_dataset, train_test_split
from FinanceSigWGANBase.utils.utils import set_seed, save_obj, get_sigwgan_experiment_dir, get_config_path, \
    plot_individual_data

os.environ['PYTHONHASHSEED'] = "0"


In [2]:
%cd "./FinanceSigWGANBase"

d:\Python\FinanceSigWGAN\FinanceSigWGANBase


In [7]:
def main(
        data_config: dict,
        dataset: str,
        experiment_dir: str,
        gan_algo: str,
        gan_config: dict,
        generator_config: dict,
        device: str = 'cpu',
        seed: Optional[int] = 0
):
    """
    Full training procedure.
    Includes: initialising the dataset / generator / GAN and training the GAN.
    """

    n_lags = data_config["n_lags"]

    # Get / prepare dataset
    x_real_rolled = get_dataset(dataset, data_config)
    x_real_rolled = x_real_rolled.to(device)
    set_seed(seed)
    print('Total data: ', list(x_real_rolled.shape))
    # exit()
    x_real_train, x_real_test = train_test_split(x_real_rolled, train_test_ratio=0.8)
    x_real_dim: int = x_real_rolled.shape[2]

    # Compute test metrics for train and test set
    test_metrics_train = get_standard_test_metrics(x_real_train)
    test_metrics_test = get_standard_test_metrics(x_real_test)

    # Get generator
    set_seed(seed)
    generator_config.update(output_dim=x_real_dim)
    g = get_generator(**generator_config).to(device)

    # Get GAN
    if gan_algo == 'SigWGAN':
        trainer = SigWGANTrainer(generator=g,
                                 x_real_rolled=x_real_rolled,
                                 test_metrics_train=test_metrics_train,
                                 test_metrics_test=test_metrics_test,
                                 **gan_config
                                 )
    else:
        raise NotImplementedError()

    # Start training
    set_seed(seed)
    trainer.fit(device=device)

    # Store relevant training results
    # save_obj(to_numpy(x_real_rolled), os.path.join(experiment_dir, 'x_real_rolled.pkl'))
    # save_obj(to_numpy(x_real_train), os.path.join(experiment_dir, 'x_real_train.pkl'))
    save_obj(to_numpy(x_real_test), os.path.join(experiment_dir, 'x_real_test.pkl'))
    save_obj(trainer.losses_history, os.path.join(experiment_dir, 'losses_history.pkl'))  # dev of losses / metrics
    save_obj(trainer.G.state_dict(), os.path.join(experiment_dir, 'generator_state_dict.pt'))
    save_obj(trainer.G, os.path.join(experiment_dir, 'generator_full_model.pt'))
    save_obj(generator_config, os.path.join(experiment_dir, 'generator_config.pkl'))

    loss_history = os.path.join(experiment_dir, 'LossHistory')
    os.makedirs(loss_history, exist_ok=True)

    plt.plot(trainer.losses_history['sig_w1_loss'], alpha=0.8)
    plt.grid()
    plt.yscale('log')
    plt.savefig(os.path.join(loss_history, 'sig_loss.png'))
    plt.close()

    plot_test_metrics(trainer.test_metrics_train, trainer.losses_history, 'train', locate_dir=loss_history)

    plot_test_metrics(trainer.test_metrics_train, trainer.losses_history, 'test', locate_dir=loss_history)

    with torch.no_grad():
        x_fake = g(1024, n_lags, device)

    for i in range(x_real_dim):
        plt.plot(to_numpy(x_fake[:400, :, i]).T, 'C%s' % i, alpha=0.1)
    plt.savefig(os.path.join(experiment_dir, 'x_fake.png'))
    plt.close()

    for i in range(x_real_dim):
        random_indices = torch.randint(0, x_real_rolled.shape[0], (400,))
        plt.plot(to_numpy(x_real_rolled[random_indices, :, i]).T, 'C%s' % i, alpha=0.1)
    plt.savefig(os.path.join(experiment_dir, 'x_real.png'))
    plt.close()

    plot_individual_data(dataset=data_config['name'], locate_dir=os.path.join(experiment_dir, 'RealDataFigure'))

    evaluate_generator(experiment_dir, batch_size=5000, )

    plot_signature(trainer.sig_w1_metric.expected_signature_mu)
    plt.savefig(os.path.join(experiment_dir, 'sig_real.png'))
    plt.close()

    plot_signature(trainer.sig_w1_metric.expected_signature_mu)
    plot_signature(compute_expected_signature(x_fake,
                                              trainer.sig_w1_metric.depth, trainer.sig_w1_metric.augmentations))
    plt.savefig(os.path.join(experiment_dir, 'sig_real_fake.png'))
    plt.close()

In [8]:
def benchmark_sigwgan(
        datasets=('BINANCE', 'STABLECOIN'),
        generators=('LogSigRNN', 'LSTM'),
        n_seeds=10,
        device='cuda:0',
):
    """ Benchmark for SigWGAN. """
    seeds = list(range(n_seeds))

    grid = itertools.product(datasets, generators, seeds)

    for dataset, generator, seed in grid:
        print(f"data:{dataset}, gen:{generator}, seed:{seed}")
        data_config = load_obj(get_config_path('', dataset))
        gan_config = load_obj(get_config_path('Trainer', 'trainer_SigWGAN'))
        generator_config = load_obj(get_config_path('Generator', 'gen_' + generator))

        experiment_dir = get_sigwgan_experiment_dir(dataset, generator, 'SigWGAN', seed)

        if not os.path.exists(experiment_dir):
            os.makedirs(experiment_dir)

        save_obj(data_config, os.path.join(experiment_dir, 'data_config.json'))
        save_obj(gan_config, os.path.join(experiment_dir, 'gen_config.json'))
        save_obj(generator_config, os.path.join(experiment_dir, 'generator_config.json'))

        if gan_config.get('augmentations') is not None:
            gan_config['augmentations'] = parse_augmentations(gan_config.get('augmentations'))

        if generator_config.get('augmentations') is not None:
            generator_config['augmentations'] = parse_augmentations(generator_config.get('augmentations'))

        if generator_config['generator_type'] == 'LogSigRNN':
            generator_config['n_lags'] = data_config['n_lags']

        save_obj(data_config, os.path.join(experiment_dir, 'data_config.pkl'))
        save_obj(gan_config, os.path.join(experiment_dir, 'gen_config.pkl'))
        save_obj(generator_config, os.path.join(experiment_dir, 'generator_config.pkl'))

        print('Training: %s' % experiment_dir.split('/')[-2:])

        main(
            dataset=dataset,
            data_config=data_config,
            device=device,
            experiment_dir=experiment_dir,
            gan_algo='SigWGAN',
            seed=seed,
            gan_config=gan_config,
            generator_config=generator_config,
        )

In [9]:
if torch.cuda.is_available():
    compute_device = 'cuda:0'
else:
    compute_device = 'cpu'

# target_dataset = os.listdir('./datasets')
# target_dataset.remove('Uniswap')
# target_dataset.append('Uniswap')
# target_dataset = ('BINANCE',)
# target_dataset = ('WrappedBitcoin',)
# target_dataset = ('STABLECOIN',)
# target_dataset = ('Uniswap',)
# target_dataset = ('BINANCE', 'STABLECOIN')
target_dataset = ('MyBinance', )
training_generators = ('LogSigRNN',)
# training_generators = ('LSTM',)
# training_generators = ('LogSigRNN', 'LSTM')
benchmark_sigwgan(datasets=target_dataset, 
                  generators=training_generators,
                  n_seeds=50,
                  device=compute_device)


data:MyBinance, gen:LogSigRNN, seed:0
Training: ['MyBinance', 'SigWGAN_LogSigRNN_0']
Use data: 
	datasets\MyBinance\BNBUSDT_1h.csv
	datasets\MyBinance\BTCUSDT_1h.csv
	datasets\MyBinance\ETHUSDT_1h.csv
	Rolled data for training, shape [67410, 24, 1]
	Example : 0.000000, 0.008248, 0.010902 ...
Total data:  [67410, 24, 1]


sig-w1 loss: 6.007526e+01: 100%|██████████| 2/2 [00:00<00:00,  3.66it/s]


data:MyBinance, gen:LogSigRNN, seed:1
Training: ['MyBinance', 'SigWGAN_LogSigRNN_1']
Use data: 
	datasets\MyBinance\BNBUSDT_1h.csv
	datasets\MyBinance\BTCUSDT_1h.csv
	datasets\MyBinance\ETHUSDT_1h.csv
	Rolled data for training, shape [67410, 24, 1]
	Example : 0.000000, 0.008248, 0.010902 ...
Total data:  [67410, 24, 1]


sig-w1 loss: 2.989285e+01: 100%|██████████| 2/2 [00:00<00:00,  4.12it/s]


data:MyBinance, gen:LogSigRNN, seed:2
Training: ['MyBinance', 'SigWGAN_LogSigRNN_2']
Use data: 
	datasets\MyBinance\BNBUSDT_1h.csv
	datasets\MyBinance\BTCUSDT_1h.csv
	datasets\MyBinance\ETHUSDT_1h.csv
	Rolled data for training, shape [67410, 24, 1]
	Example : 0.000000, 0.008248, 0.010902 ...
Total data:  [67410, 24, 1]


...

sig-w1 loss: 1.382948e+02: 100%|██████████| 2/2 [00:00<00:00,  2.34it/s]


data:MyBinance, gen:LogSigRNN, seed:48
Training: ['MyBinance', 'SigWGAN_LogSigRNN_48']
Use data: 
	datasets\MyBinance\BNBUSDT_1h.csv
	datasets\MyBinance\BTCUSDT_1h.csv
	datasets\MyBinance\ETHUSDT_1h.csv
	Rolled data for training, shape [67410, 24, 1]
	Example : 0.000000, 0.008248, 0.010902 ...
Total data:  [67410, 24, 1]


sig-w1 loss: 5.076521e+01: 100%|██████████| 2/2 [00:00<00:00,  2.37it/s]


data:MyBinance, gen:LogSigRNN, seed:49
Training: ['MyBinance', 'SigWGAN_LogSigRNN_49']
Use data: 
	datasets\MyBinance\BNBUSDT_1h.csv
	datasets\MyBinance\BTCUSDT_1h.csv
	datasets\MyBinance\ETHUSDT_1h.csv
	Rolled data for training, shape [67410, 24, 1]
	Example : 0.000000, 0.008248, 0.010902 ...
Total data:  [67410, 24, 1]


sig-w1 loss: 6.013242e+01: 100%|██████████| 2/2 [00:00<00:00,  2.35it/s]
